# Example: Advanced model customisation

This notebook covers an example on how to find functionality in hardware for a more complex task that requires more advanced usage of custom models. It will cover an example with the famous LeNet neural network architecture for the classical classification benchmark task of digit recognition with the Modified National Institute of Standards and Technology (MNIST) dataset.  


**Prerequisites:**
- You have a surrogate model from a DNPU model that is still connected to an NI device in the lab
    
    

In [3]:
from IPython.display import Image
from IPython.core.display import HTML

# Image from https://www.codexa.net/cnn-mnist-keras-beginner/
Image(url= "4.finding_functionality/cnn_main.png")

## 1. Having a look at the data

In [4]:
from torchvision.datasets import MNIST

# Downloading dataset in ./tmp/data
dataset_example = MNIST('./tmp/data', train=True, download=True)

### 1.1 Images

In [5]:
dataset_example

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./tmp/data
    Split: Train

In [6]:
first_dataset_element = dataset_example[0]

In [7]:
type(first_dataset_element)

tuple

In [8]:
first_dataset_element

(<PIL.Image.Image image mode=L size=28x28>, 5)

In [9]:
first_img = first_dataset_element[0]

In [10]:
first_img

In [11]:
import numpy as np

first_img_np = np.array(first_img)

In [12]:
first_img_np.shape

(28, 28)

In [13]:
first_img_np.min()

0

In [14]:
first_img_np.max()

255

In [15]:
first_img_np

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,
         18,  18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,
          0,   0],
       [  

In [16]:
first_img_np/255

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

### 1.2 Image modifications

In [17]:
labelled_values = []
desired_label = 5
for i in range(len(dataset_example)):
    current = dataset_example[i]
    if current[1] == desired_label:
        labelled_values.append(np.array(current[0]))

labelled_values = np.array(labelled_values)

In [18]:
labelled_values.shape

(5421, 28, 28)

In [1]:
import matplotlib.pyplot as plt
import numpy as np
plt.imshow(labelled_values.mean(0),cmap='gray')
plt.show()

NameError: name 'labelled_values' is not defined

### 1.3 Labels and One-hot encoding

In [ ]:
first_label = first_dataset_element[1]

In [ ]:
first_label

In [1]:
first_label_one_hot = np.zeros(10)
first_label_one_hot[first_label] = 1
first_label_one_hot

NameError: name 'np' is not defined

Note that the model should also produce outputs that are a vector with the same dimension as the number of labels. In this case 10

## 2. Understanding loss functions

The cross entropy loss function can be decomposed into Softmax and Negative Likelihood. We can have a look at these concepts individually.

### 2.1 Softmax

We want to represent the raw output of our model (logits) as a probability, for which it would need to satisfy the following conditions:
- Each element in $\hat{y_i}$ is in a range from 0 to 1. 
- All the elements of the output prediction vector $\hat{y_i}$ add to 1.
- It maintains this ratio when translating it into a probability. I.e. if a if one value in $\hat{y_i}$ is greater, it should maintain its proportion. 

In principle, we could try to satisfy these conditions by dividing each of the elements in the output vector prediction $\hat{y_i}$ by the sum of all the elements in the same vector. 

$ f({y_i}) = \frac{\hat{y_{ij}}}{\sum_{j=0}^{n} \hat{y_{ij}}} $

In [1]:
y_i_hat = np.array([3,2,1])

NameError: name 'np' is not defined

In [ ]:
f = []
for i in range(len(y_i_hat)):
    f.append(y_i_hat[i] / y_i_hat.sum())
    print(f[-1])

In [ ]:
np.array(f).sum()

However, if we have negative values, we could end with NaN values. In order to maintain numerical stability, we can turn all negative values into positive ones we use the function $e^x$. Making the softmax function formula be: 

$  f({y_i}) = \frac{e^{\hat{y_{ij}}}}{\sum_{j=0}^{n} e^{\hat{y_{ij}}}} $

In [4]:

f = []
for i in range(len(y_i_hat)):
    f.append( np.exp(y_i_hat[i]) / np.exp(y_i_hat).sum())
    print(f[-1])

NameError: name 'y_i_hat' is not defined

In [5]:

np.array(f).sum()

0.0

Where n is the number of classes in the dataset. $\hat{y_i}$ is a prediction vector of the same size as the number of classes. The model should maintain the same vector lenghth as the target values. Remember that we have already seen an example of how target vectors look like for the number 5:

In [ ]:
first_label_one_hot

Finally, the division can be made more numerically stable (avoiding division by zero), when transformed into the log domain, where it would be a substraction. Therefore, 
it is not uncommon to use $log(f(y_i))$ instead. This other function is called log softmax.

In [ ]:
# Source from https://www.youtube.com/watch?v=RC_A9Tu99y4

### 2.2 Maximum likelihood

The goal of maximum likelihood is to find a distribution that maximises the likelihood of observing the outputs that are measured. Therefore, it assigns higher probabilities to models that are able to represent probabilities based on what was observed and picks the model that gives existing labels the highest probability. 

To understand this better, let us assume that we have a simple perceptron  $\hat{y}$ = $\sigma$(W*x + b) that classifies four points (two blue and two red). Our particular prediction $\hat{y}$ is equal to the probability of a particular point being labelled correctly $P$(blue) = $\sigma$(W*x + b). Let us assume that you have two different models, giving you different probabilities for the same points. The first model is represented on the left of the image, and the second model is represented on the right of the image below. The images show the probability of being red in red, and the probability of being blue in blue. Note that the probability of being blue is one minus the probability of being red. Assuming that the points are independent events, then the probability for the whole arrangement is the product of the probabilities of each point.

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML

# Image and lesson from https://www.youtube.com/watch?v=6nUUeQ9AeUA
Image(url= "4.finding_functionality/likelihood.png")

In [ ]:
-(np.log(0.6) + np.log(0.2) + np.log(0.1) + np.log(0.7))

In [ ]:
-(np.log(0.7) + np.log(0.9) + np.log(0.8) + np.log(0.6))

### 2.3 Binary cross entropy

We want to maximise the likelihood of a model, but for this we need to multiply many points, which is inconvenient when we have huge datasets. In order to make products into sums, we use log functions. Probabilities are expressed in a range from 0 to 1, but on log scale these values are negative, so we take the negative log of the probabilities. Lower cross entropy is better than higher. The final formula for cross entropy is:

$- \sum_{i=0}^{m} y_i . ln(p_i) + (1 - y_i) . ln(1 - p_i)$

Where m is the number of elements in the dataset.

### 2.4 Multi-class cross entropy

In a very similar way, this concept can be extended to multi classes.

$- \sum_{j=0}^{n} \sum_{i=0}^{m} y_{ij} . ln(p_{ij})$

Where n is the number of classes in the dataset. $\hat{y_i}$ is a prediction vector of the same size as the number of classes. The model should maintain the same vector lenghth as the target values. Remember that we have already seen an example of how target vectors look like for the number 5:

In [ ]:
first_label_one_hot

## 3. Preparing the data

## 4. Preparing the training loop

In [29]:
import pytorch_lightning as pl
from torch.utils.data import random_split

from torch.jit.annotations import Optional
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader


class MNISTDataModule(pl.LightningDataModule):
    def __init__(self,
                 data_dir: str = "./tmp/data",
                 batch_size: int = 32,
                 num_workers=4,
                 pin_memory=False,
                 split=[55000, 5000]):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.pin_memory = pin_memory
        self.split = split

    def prepare_data(self):
        # download
        MNIST(self.data_dir, train=True, download=True)
        MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage: Optional[str] = None):
        if stage == 'fit' or stage is None:
            mnist_full = MNIST(self.data_dir,
                               train=True,
                               transform=transforms.ToTensor())
            self.mnist_train, self.mnist_val = random_split(
                mnist_full, self.split)
        if stage == 'test' or stage is None:
            self.mnist_test = MNIST(self.data_dir,
                                    train=False,
                                    transform=transforms.ToTensor())

    def train_dataloader(self):
        return DataLoader(self.mnist_train,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          shuffle=True,
                          pin_memory=self.pin_memory)

    def val_dataloader(self):
        return DataLoader(self.mnist_val,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          shuffle=False,
                          pin_memory=self.pin_memory)

    def test_dataloader(self):
        return DataLoader(self.mnist_test,
                          batch_size=self.batch_size,
                          num_workers=self.num_workers,
                          shuffle=False,
                          pin_memory=self.pin_memory)

In [30]:
import pytorch_lightning as pl

import torch
import torch.nn.functional as F

from torchmetrics.functional.classification.accuracy import accuracy

from pytorch_lightning.callbacks import ModelCheckpoint


class TrainerMNIST(pl.LightningModule):
    def __init__(self, model, batch_size, learning_rate=1e-3):
        super(TrainerMNIST, self).__init__()
        self.name = 'mnist_trainer'
        self.model = model
        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return opt

    def forward(self, x):
        x = self.model(x)
        x = F.log_softmax(x, dim=1)
        return x
    
    def shared_step(self, batch):
        x, y = batch
        preds = self(x)
        loss = F.nll_loss(preds, y)
        acc = accuracy(preds, y)
        return loss, acc

    def training_step(self, batch, batch_idx):
        loss, acc = self.shared_step(batch)
        self.log('loss_epoch', {'train': loss}, on_step=False, on_epoch=True)
        self.log('acc_epoch', {'train': acc}, on_step=False, on_epoch=True)
        self.log('train_loss', loss, prog_bar=False)
        self.log('train_acc', acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, acc = self.shared_step(batch)
        self.log('loss_epoch', {'val': loss}, on_step=False, on_epoch=True)
        self.log('acc_epoch', {'val': acc}, on_step=False, on_epoch=True)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, acc = self.shared_step(batch)
        self.log('loss_epoch', {'test': loss}, on_step=False, on_epoch=True)
        self.log('acc_epoch', {'test': acc}, on_step=False, on_epoch=True)
        self.log('test_loss', loss, prog_bar=True)
        self.log('test_acc', acc, prog_bar=True)
        return loss

    # Allow brains-py to do constrained optimisation
    def training_step_end(self, outputs):
        if 'constraint_weights' in dir(self.model):
            self.model.constraint_weights()
        return outputs
    
    # Make pretty the progress bar
    def get_progress_bar_dict(self):
        # don't show the version number
        items = super().get_progress_bar_dict()
        items.pop("v_num", None)
        return items


## 5. Creating a vanilla pytorch model

A custom model can be created in pytorch by implementing a child class of nn.Module

In [31]:
from IPython.display import Image
from IPython.core.display import HTML

Image(url= "4.finding_functionality/lenet3.jpg")

In [32]:
from torch import nn

class FlattenView(nn.Module):
    def __init__(self):
        super(FlattenView, self).__init__()

    def forward(self, x):

        #return x.view(x.shape[0], -1)
        return x.flatten(1, -1)

In [33]:
from torch import nn

class LeNet(nn.Module):
    
    # Constructor: Initialisation stage
    def __init__(self, mode='nn'):
        super(LeNet, self).__init__()
        self.name = 'LeNet'

        # First convolution layer
        #                     (in_channels, out_channels, kernel_size)
        self.conv1 = nn.Conv2d(1          ,6            , 3)
        
        
        # Second convolution layer
        #                     (in_channels, out_channels, kernel_size)
        self.conv2 = nn.Conv2d(6          ,16           , 3)
        
        # Fully connected layer
        self.fc = nn.Sequential(FlattenView(), 
                                nn.Linear(400, 120),
                                nn.ReLU(), 
                                nn.Linear(120, 84), 
                                nn.ReLU(),
                                nn.Linear(84, 10))
        # Activation function
        self.act = nn.ReLU()
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)
        
        x = self.fc(x)
        
        return x # Logits

## 6. Training the vanilla model

### Hyperparemeters

In [34]:
batch_size = 64
lr = 1e-3

### Seed

In [35]:
from pytorch_lightning import seed_everything

seed = 2508835240
seed = seed_everything(seed, workers=True)

Global seed set to 2508835240


In [36]:
mnist = MNISTDataModule(batch_size=batch_size)

In [37]:
checkpoint_callback = ModelCheckpoint(
        monitor='val_acc', filename='sample-mnist-{val_acc:.3f}', mode='max')

In [38]:
model = LeNet()

In [39]:
model_wrapper = TrainerMNIST(model, batch_size=batch_size, learning_rate=lr)

In [42]:
trainer = pl.Trainer(gpus=0,max_epochs=5,
    callbacks=[checkpoint_callback],
    deterministic=True)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/unai/anaconda3/envs/bspy/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1294: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=lightning_logs

In [43]:
trainer.fit(model_wrapper, datamodule=mnist)

/home/unai/anaconda3/envs/bspy/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name  | Type  | Params
--------------------------------
0 | model | LeNet | 60.1 K
--------------------------------
60.1 K    Trainable params
0         Non-trainable params
60.1 K    Total params
0.240     Total estimated model params size (MB)


Global seed set to 2508835240


/home/unai/anaconda3/envs/bspy/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1046: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


## 6. Modifying LeNet

In [44]:
from IPython.display import Image
from IPython.core.display import HTML

Image(url= "4.finding_functionality/lenetmodified.jpg")

In [ ]:
from torch import nn

class LeNetModified(nn.Module):
    
    # Constructor: Initialisation stage
    def __init__(self, mode='nn'):
        super(LeNetModified, self).__init__()
        self.name = 'LeNetModified'

        # First convolution layer
        #                     (in_channels, out_channels, kernel_size)
        self.conv1 = nn.Conv2d(1          ,6            , 3)
        self.bn1 = nn.BatchNorm2d(6) # Same as out channels
        
        # Second convolution layer
        #                     (in_channels, out_channels, kernel_size)
        self.conv2 = nn.Conv2d(6          ,16           , 3)
        self.bn2 = nn.BatchNorm2d(16) # Same as out channels
        
        # Fully connected layer
        self.fc = nn.Sequential(FlattenView(), 
                                nn.Linear(400, 120),
                                nn.ReLU(), 
                                nn.Linear(120, 84), 
                                nn.ReLU(),
                                nn.Linear(84, 10))
        # Activation function
        self.act = nn.Sigmoid() # Changed activation function
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x) # New line
        x = self.act(x) 
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x) # New line
        x = self.act(x)
        x = self.pool(x)
        
        x = self.fc(x)
        
        return x # Logits

In [ ]:
model_modified = LeNetModified()
model_modified_wrapper = TrainerMNIST(model_modified, batch_size=batch_size, learning_rate=lr)
trainer = pl.Trainer(gpus=1,max_epochs=5,
    callbacks=[checkpoint_callback],
    deterministic=True)
trainer.fit(model_modified_wrapper, datamodule=mnist)

For this exercise, the Genetic Algorithm will be used, as this is a fast way to explore functionality on devices directly on-chip. The particularities of each config is explained below.

## 7. Using convolutions with DNPUs

In [45]:
def get_node_configs():
    configs = {}
    configs["model_data_path"] = "./training_data.pt"  # "./tmp/models/training_data_new_without_clipping.pt"  #"./tmp/models/training_data.pt"
    configs["processor_type"] = "simulation"
    configs["waveform"] = {}
    configs["waveform"]["plateau_length"] = 1
    configs["waveform"]["slope_length"] = 0
    return configs

In [ ]:
from torch import nn
from brainspy.processors.processor import Processor
from brainspy.processors.modules.conv import DNPUConv2d

class LeNetDNPU(nn.Module):
    
    # Constructor: Initialisation stage
    def __init__(self, mode='nn'):
        super(LeNetDNPU, self).__init__()
        self.name = 'LeNetDNPU'
        
        ###############################################################
        # Declare a processor
        configs = get_node_configs()
        model_data = torch.load(configs['model_data_path'])
        self.processor_base = Processor(configs, model_data['info'],
                         model_data['model_state_dict'])
        
        # Select data input indices of each DNPU
        data_input_indices_list = [[2, 3, 4]] * 3
        ###############################################################
        
        # First convolution layer
        #            nn.Conv2d(in_channels, out_channels, kernel_size)
        self.conv1 = DNPUConv2d(self.processor_base, data_input_indices_list,
                               1          ,6            , 3)
        self.bn1 = nn.BatchNorm2d(6) # Same as out channels
        
        # Second convolution layer
        #            nn.Conv2d(in_channels, out_channels, kernel_size)
        self.conv2 = DNPUConv2d(self.processor_base, data_input_indices_list,
                               6          ,16            , 3)
        self.bn2 = nn.BatchNorm2d(16) # Same as out channels
        
        # Fully connected layer
        self.fc = nn.Sequential(FlattenView(), 
                                nn.Linear(400, 120),
                                nn.ReLU(), 
                                nn.Linear(120, 84), 
                                nn.ReLU(),
                                nn.Linear(84, 10))
        # Activation function
        self.act = nn.Sigmoid() # Changed activation function
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2)
        
        ###############################################################
        self.conv1.add_input_transform([0, 1])
        self.conv2.add_input_transform([0, 1])
        ###############################################################
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x) # New line
        x = self.act(x) 
        x = self.pool(x)
        
        x = self.conv2(x)
        x = self.bn2(x) # New line
        x = self.act(x)
        x = self.pool(x)
        
        x = self.fc(x)
        
        return x # Logits
    
    ###############################################################
    def swap(self, configs, info):
        self.processor_base.swap(configs, info)
    ###############################################################
        
        

In [ ]:
batch_size = 2

model_modified = LeNetDNPU().to('cpu')
model_modified_wrapper = TrainerMNIST(model_modified, batch_size=batch_size, learning_rate=lr).to('cpu')

trainer = pl.Trainer(gpus=0, max_epochs=5,
    callbacks=[checkpoint_callback],
    deterministic=True)

trainer.fit(model_modified_wrapper, datamodule=mnist)

## 8. Testing convolutions on real DNPU hardware

In [47]:
import torch

results = torch.load('4.finding_functionality/sample-mnist-val_acc=0.98-v2.ckpt')

Exception ignored in: <function tqdm.__del__ at 0x7fecaabcc0d0>
Traceback (most recent call last):
  File "/home/unai/anaconda3/envs/bspy/lib/python3.8/site-packages/tqdm-4.51.0-py3.8.egg/tqdm/std.py", line 1128, in __del__
    self.close()
  File "/home/unai/anaconda3/envs/bspy/lib/python3.8/site-packages/tqdm-4.51.0-py3.8.egg/tqdm/notebook.py", line 266, in close
    self.sp(close=True)
AttributeError: 'tqdm' object has no attribute 'sp'


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
results.keys()

In [ ]:
results['state_dict']

In [ ]:
my_model = LeNetDNPU()
my_model.load_state_dict(results['state_dict'])

In [ ]:
processor_configs = {
    "processor_type" : 'cdaq_to_cdaq', # There are four different processor types: 'simulation', 'cdaq_to_cdaq', 'cdaq_to_nidaq', 'simulation_debug'. The first 
    'input_indices': [3,6], # It specifies what indices will be taken as data input. These correspond to the indices of the activation channels inside the instruments setup configuration. 
    'waveform': { # The waveform determines the number of points that will be used to represent a single point. More information about waveforms can be found at Section 5.1 Waveforms: https://github.com/BraiNEdarwin/brains-py/wiki/A.-Introduction
        'plateau_length' : 30, 
        'slope_length' : 30
    }
}

In [ ]:
driver_configs = {
        'inverted_output': True, # Whether if the op-amp circuit to amplify the output of the DNPU applies an inversion or not.
        'amplification': [41], # Indicates the amplification correction factor that will be applied to obtain the real current measurement from the setup. More information about this can be found in Section 5.3 of the introduction of brains-py wiki: https://github.com/BraiNEdarwin/brains-py/wiki/A.-Introduction
        'instruments_setup':{ 
            'multiple_devices': False, # Indicates whether if the setup is using a PCB with multiple devices or not.
            'trigger_source': 'cDAQ1/segment1', # Triggering signal to be sent for reading synchronisation. You can check for this signal for your setup on the NIMax app.
            'average_io_point_difference': True, # If number of points read from the device is different from the number of points written to the device due to a difference in their
                                                # sampling frequencies, this variable indicates if there should be an averaging so that the input and output have the same length.
            'activation_instrument': 'cDAQ1Mod3', # Main module used for sending voltage signals to the DNPU 
            'activation_sampling_frequency': 1000, # Number of samples that will be written to the DNPU in one second.
            'activation_channels': [6,0,1,5,2,4,3], # Channels of the module that will be used for sending signals the device. 
            'activation_voltage_ranges': # Maximum minimum and maximum voltage ranges that will be allowed to be sent to the DNPU, per electrode. Dimensions (Electrode_no, 2)
              [
                [-0.7,1.2],[-0.3,0.4],[-0.7,1.2],[-0.8,1.1],[-0.6,1.1],[-0.3,0.5],[-0.7,1.1]
              ],
            'readout_instrument': 'cDAQ1Mod4', # Main module used for receiving voltage signals from the DNPU (after the op-amp) 
            'readout_sampling_frequency': 1000, # Number of samples that will be read from the DNPU in one second
            'readout_channels': [0], # Channels of the module that will be used for reading signals from the device. 
            'activation_channel_mask': [1, 1, 1, 1, 1, 1, 1] # Whether if all the channels connected to the device electrodes would be used or not. 1 for enabling its use, 0 for disabling it. When disabled, the channels will not be declared.
        }
    
    }

processor_configs['driver'] = driver_configs

In [ ]:
model_data = torch.load("./training_data.pt")

In [ ]:
my_model.swap(processor_configs, model_data['info'])

In [ ]:
my_model(data)

In [46]:
from IPython.display import Image
from IPython.core.display import HTML

Image(url= "4.finding_functionality/processor.png")

 - NNModel: Pytorch model (trained smg)
 - SoftwareProcessor:
    * Noise
    * Amplification
    * Clipping
 - Processor: 
    * Handle changes from SurrogateModel to HardwareModel    
 - DNPU:
    * Declares control voltages as learnable parameters
 - Driver: Connect to NISetup    
 - Hardware model:
    * Apply slopes
    * Transform to numpy
    * Apply mask
    * Transform to pytorch


